# Guess Who?

Aswin van Woudenberg (https://www.aswinvanwoudenberg.com | https://github.com/afvanwoudenberg)

In [1]:
import pandas as pd
import numpy as np
import re

from ipywidgets import GridspecLayout, HTML, VBox, HBox, Button, Label
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.DataFrame({
    # hair style
    'hair_partition': [0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0], 
    'curly_hair':     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
    'hat':            [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'bald':           [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
    'long_hair':      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    # hair color
    'ginger_hair':    [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'white_hair':     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0],
    'brown_hair':     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0],
    'blond_hair':     [0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'black_hair':     [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
    # facial attributes
    'big_mouth':      [1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
    'big_nose':       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
    'red_cheeks':     [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0],
    'blue_eyes':      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    'sad_looking':    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    # facial hair
    'facial_hair':    [1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
    'moustache':      [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    'beard':          [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    # other
    'glasses':        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
    'earrings':      [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'female':         [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    # names
    'name':           ['alex', 'alfred', 'anita', 'anne', 'bernard', 'bill', 'charles', 'claire', 
                       'david', 'eric', 'frans', 'george', 'herman', 'joe', 'maria', 'max', 'paul', 
                       'peter', 'philip', 'richard', 'robert', 'sam', 'susan', 'tom']
})

In [3]:
column_description = {
    "hair_partition": "Does your character have a visible hair partition?",
    "curly_hair": "Does your character have curly hair?",
    "hat": "Does your character wear a hat?",
    "bald": "Is your character bald?",
    "long_hair": "Does your character have long hair?",
    "ginger_hair": "Does your character have ginger hair?",
    "white_hair": "Does your character have white hair?",
    "brown_hair": "Does your character have brown hair?",
    "blond_hair": "Does your character have blond hair?",
    "black_hair": "Does your character have black hair?",
    "big_mouth": "Does your character have a big mouth?",
    "big_nose": "Does your character have a big nose?",
    "red_cheeks": "Does your character have red cheeks?",
    "blue_eyes": "Does your character have blue eyes?",
    "sad_looking": "Does your character look sad?",
    "facial_hair": "Does your character have facial hair?",
    "moustache": "Does your character have a moustache?",
    "beard": "Does your character have a beard?",
    "glasses": "Does your character wear glasses?",
    "earrings": "Does your character wear earrings?",
    "female": "Is your character female?"
}

In [4]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [5]:
feature_names = list(df.columns)[:-1]
target_name = df.columns[-1]

In [6]:
clf = DecisionTreeClassifier(criterion = "entropy")
clf = clf.fit(X, y)

In [7]:
grid = GridspecLayout(3, 8)
grid.width = "1400px"

for i in range(3):
    for j in range(8):
        grid[i, j] = HTML(value="<img src='https://guesswhocharacters.info/imgs/{}.jpeg' width=162 height=237>".format(str(df.name[i*8+j])))

label = Label(value="Put the questions and answer here")
button_yes = Button(description="Yes")
button_no = Button(description="No")
button_play_again = Button(description="Play again")
hbox = HBox([label, button_yes, button_no, button_play_again])
vbox = VBox([grid, hbox])

In [8]:
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
value = clf.tree_.value

# Start at root
node_id = 0

In [9]:
def update_gui():
    if children_left[node_id] == children_right[node_id]:
        label.value = "You character is " + y[np.argmax(value[node_id])].capitalize()
        button_yes.layout.display = 'none'
        button_no.layout.display = 'none'
        button_play_again.layout.display = 'block'
    else:
        label.value = column_description[df.columns[feature[node_id]]]
        button_yes.layout.display = 'block'
        button_no.layout.display = 'block'
        button_play_again.layout.display = 'none'

def show_all_cards():
    for i in range(3):
        for j in range(8):
            grid[i, j].layout.visibility = 'visible'

def hide_cards(attribute, val):
    for i in range(3):
        for j in range(8):
            if df[attribute][i*8+j] == val:
                grid[i, j].layout.visibility = 'hidden'
        
def on_button_yes_clicked(b):
    global node_id
    hide_cards(df.columns[feature[node_id]], 0)
    node_id = children_right[node_id]
    update_gui()

def on_button_no_clicked(b):
    global node_id
    hide_cards(df.columns[feature[node_id]], 1)
    node_id = children_left[node_id]
    update_gui()

def on_button_play_again_clicked(b):
    global node_id
    show_all_cards()
    node_id = 0
    update_gui()
    
button_yes.on_click(on_button_yes_clicked)
button_no.on_click(on_button_no_clicked)
button_play_again.on_click(on_button_play_again_clicked)

update_gui()

In [10]:
display(vbox)